In [1]:
!pip install tensorflow>=2.13.0 transformers matplotlib seaborn scikit-learn

# Connect to my google drive
from google.colab import drive
drive.mount('/content/drive')

# copy files from google drive to project
!cp '/content/drive/MyDrive/Uni/BD/part-00000.snappy.parquet' '/content/'
!cp '/content/drive/MyDrive/Uni/BD/part-00001.snappy.parquet' '/content/'
!cp '/content/drive/MyDrive/Uni/BD/part-00002.snappy.parquet' '/content/'
!cp '/content/drive/MyDrive/Uni/BD/part-00003.snappy.parquet' '/content/'

Mounted at /content/drive


### Now

In [1]:
"""
Deep Learning Models for Sentiment Analysis - GPU/TPU Optimized for Google Colab
PIPELINE-COMPATIBLE VERSION - Works with existing MICAP preprocessing pipeline
Implements LSTM, CNN, and Transformer models with CUDA/TPU acceleration

Author: AI Assistant
Date: 2025-01-20
Dependencies: tensorflow>=2.13.0, pandas, numpy, pyspark (for data loading)
Environment: Google Colab with GPU/TPU runtime
Pipeline: Reads from preprocessed parquet files created by MICAP pipeline
"""

import os
import numpy as np
import pandas as pd
import logging
import time
from typing import Dict, List, Tuple, Optional, Union
import json
import warnings
from pathlib import Path

# TensorFlow imports with GPU optimization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.mixed_precision import LossScaleOptimizer

# Additional ML libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# PySpark imports for reading existing pipeline data
try:
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col
    PYSPARK_AVAILABLE = True
except ImportError:
    PYSPARK_AVAILABLE = False
    logging.warning("PySpark not available. Will use pandas for data loading.")

tf.config.optimizer.set_jit(True)      # XLA compilation
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

# Configure warnings and logging
warnings.filterwarnings('ignore', category=FutureWarning)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)


class PipelineDataLoader:
    """
    Loads data from MICAP preprocessing pipeline (parquet files)
    Maintains compatibility with existing feature engineering
    """

    def __init__(self):
        """Initialize pipeline data loader."""
        self.spark = None
        if PYSPARK_AVAILABLE:
            self._init_spark_session()

    def _init_spark_session(self):
        """Initialize Spark session for reading parquet files."""
        try:
            self.spark = (SparkSession.builder
                         .appName("ColabDataLoader")
                         .master("local[*]")
                         .config("spark.driver.memory", "64g")
                         .config("spark.sql.shuffle.partitions", "200")
                         .getOrCreate())
            self.spark.sparkContext.setLogLevel("WARN")
            logger.info("Spark session created for pipeline data loading")
        except Exception as e:
            logger.error(f"Failed to create Spark session: {e}")
            self.spark = None

    def load_pipeline_data(self, data_path: str,
                          sample_fraction: float = 1.0,
                          use_spark: bool = True) -> pd.DataFrame:
        """
        Load preprocessed data from MICAP pipeline.

        Args:
            data_path: Path to processed parquet file or directory
            sample_fraction: Fraction of data to use
            use_spark: Whether to use Spark for loading (fallback to pandas)

        Returns:
            pd.DataFrame: Loaded and sampled data
        """
        logger.info(f"Loading pipeline data from: {data_path}")

        if use_spark and self.spark and PYSPARK_AVAILABLE:
            return self._load_with_spark(data_path, sample_fraction)
        else:
            return self._load_with_pandas(data_path, sample_fraction)

    def _load_with_spark(self, data_path: str, sample_fraction: float) -> pd.DataFrame:
        """Load data using Spark (maintains original pipeline compatibility)."""
        logger.info("Loading data with Spark...")

        try:
            # Read parquet file(s)
            df = self.spark.read.parquet(data_path)

            # Sample if requested
            if sample_fraction < 1.0:
                df = df.sample(sample_fraction, seed=42)

            # Convert to pandas for TensorFlow compatibility
            # Use efficient streaming for large datasets
            pandas_df = self._spark_to_pandas_efficient(df)

            logger.info(f"Loaded {len(pandas_df)} records with Spark")
            return pandas_df

        except Exception as e:
            logger.error(f"Spark loading failed: {e}")
            logger.info("Falling back to pandas...")
            return self._load_with_pandas(data_path, sample_fraction)

    def _load_with_pandas(self, data_path: str, sample_fraction: float) -> pd.DataFrame:
        """Load data with pandas (fallback method)."""
        logger.info("Loading data with pandas...")

        try:
            # Try reading as parquet first
            if data_path.endswith('.parquet') or os.path.isdir(data_path):
                df = pd.read_parquet(data_path)
            else:
                # Fallback to CSV
                df = pd.read_csv(data_path)

            # Sample if requested
            if sample_fraction < 1.0:
                df = df.sample(frac=sample_fraction, random_state=42)

            logger.info(f"Loaded {len(df)} records with pandas")
            return df

        except Exception as e:
            logger.error(f"Failed to load data: {e}")
            raise

    def _spark_to_pandas_efficient(self, spark_df, batch_size: int = 50000) -> pd.DataFrame:
        """Efficiently convert Spark DataFrame to pandas using streaming."""
        try:
            # Try direct conversion for smaller datasets
            if spark_df.count() < batch_size:
                return spark_df.toPandas()

            # Stream large datasets in batches
            logger.info("Streaming large dataset in batches...")
            parts = []
            for batch in spark_df.toLocalIterator(batch_size):
                batch_df = pd.DataFrame(list(batch), columns=spark_df.columns)
                parts.append(batch_df)

            return pd.concat(parts, ignore_index=True)

        except Exception as e:
            logger.warning(f"Streaming failed, using direct conversion: {e}")
            return spark_df.toPandas()

    def validate_pipeline_features(self, df: pd.DataFrame) -> bool:
        """
        Validate that the DataFrame contains expected pipeline features.

        Args:
            df: DataFrame to validate

        Returns:
            bool: True if valid pipeline data
        """
        # Expected features from MICAP pipeline
        required_features = [
            'text', 'sentiment', 'text_processed',
            'text_length', 'processed_length', 'token_count'
        ]

        # Optional but expected features
        expected_features = [
            'vader_compound', 'vader_positive', 'vader_negative', 'vader_neutral',
            'emoji_sentiment', 'exclamation_count', 'question_count',
            'uppercase_ratio', 'punctuation_density',
            'hour_sin', 'hour_cos', 'is_weekend'
        ]

        # Check required features
        missing_required = [f for f in required_features if f not in df.columns]
        if missing_required:
            logger.error(f"Missing required pipeline features: {missing_required}")
            return False

        # Log available optional features
        available_optional = [f for f in expected_features if f in df.columns]
        logger.info(f"Available pipeline features: {len(available_optional)}/{len(expected_features)}")

        return True

    def prepare_features_for_training(self, df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, Dict]:
        """
        Prepare features from pipeline data for deep learning training.

        Args:
            df: DataFrame with pipeline features

        Returns:
            Tuple of (text_sequences, labels, feature_info)
        """
        logger.info("Preparing pipeline features for training...")

        # Validate data
        if not self.validate_pipeline_features(df):
            raise ValueError("Invalid pipeline data structure")

        # Use processed text for deep learning (already cleaned by pipeline)
        text_column = 'text_processed' if 'text_processed' in df.columns else 'text'
        texts = df[text_column].fillna('').astype(str)
        labels = df['sentiment'].values

        # Extract numeric features created by pipeline
        numeric_features = []
        feature_names = []

        # Basic text features
        if 'text_length' in df.columns:
            numeric_features.append(df['text_length'].fillna(0))
            feature_names.append('text_length')

        if 'token_count' in df.columns:
            numeric_features.append(df['token_count'].fillna(0))
            feature_names.append('token_count')

        # VADER sentiment features
        vader_features = ['vader_compound', 'vader_positive', 'vader_negative', 'vader_neutral']
        for feature in vader_features:
            if feature in df.columns:
                numeric_features.append(df[feature].fillna(0))
                feature_names.append(feature)

        # Emoji and text statistics
        text_stat_features = ['emoji_sentiment', 'exclamation_count', 'question_count',
                             'uppercase_ratio', 'punctuation_density']
        for feature in text_stat_features:
            if feature in df.columns:
                numeric_features.append(df[feature].fillna(0))
                feature_names.append(feature)

        # Temporal features
        temporal_features = ['hour_sin', 'hour_cos', 'is_weekend']
        for feature in temporal_features:
            if feature in df.columns:
                numeric_features.append(df[feature].fillna(0))
                feature_names.append(feature)

        # Combine numeric features
        if numeric_features:
            numeric_array = np.column_stack(numeric_features)
            logger.info(f"Extracted {len(feature_names)} numeric features: {feature_names}")
        else:
            numeric_array = None
            logger.warning("No numeric features found in pipeline data")

        feature_info = {
            'text_column': text_column,
            'numeric_features': feature_names,
            'numeric_shape': numeric_array.shape if numeric_array is not None else None,
            'text_samples': len(texts),
            'label_distribution': pd.Series(labels).value_counts().to_dict()
        }

        logger.info(f"Feature preparation completed: {feature_info}")

        return texts.values, labels, numeric_array, feature_info

class TestDataEvaluator:
    """
    Evaluates trained models on test dataset with TPU optimization
    Designed for Google Colab TPU v6e1 with 173GB RAM
    """
    
    def __init__(self, env_manager: GPUEnvironmentManager, data_loader: PipelineDataLoader):
        """
        Initialize test evaluator with environment and data loader.
        
        Args:
            env_manager: GPU/TPU environment manager
            data_loader: Pipeline data loader instance
        """
        self.env_manager = env_manager
        self.data_loader = data_loader
        self.test_batch_size = self._calculate_optimal_test_batch_size()
        
    def _calculate_optimal_test_batch_size(self) -> int:
        """Calculate optimal batch size for TPU v6e1 with 173GB RAM"""
        # TPU v6e1 can handle very large batches
        if self.env_manager.device_type == 'TPU':
            # Use larger batch size for TPU with 173GB RAM
            return 65536  # 64K batch size for maximum throughput
        elif self.env_manager.device_type == 'GPU':
            return 16384
        else:
            return 1024
            
    def load_test_data(self, test_path: str = '/content/drive/MyDrive/data/raw/testdata.manual.2009.06.14.csv') -> pd.DataFrame:
        """
        Load test dataset from CSV file.
        
        Args:
            test_path: Path to test CSV file
            
        Returns:
            pd.DataFrame: Loaded test data
        """
        logger.info(f"Loading test data from: {test_path}")
        
        try:
            # Define column names for Sentiment140 test data
            column_names = ['polarity', 'tweet_id', 'date', 'query', 'user', 'text']
            
            # Load CSV with proper encoding
            test_df = pd.read_csv(
                test_path,
                names=column_names,
                encoding='latin-1',
                header=None
            )
            
            # Convert polarity (0=negative, 4=positive) to binary sentiment
            test_df['sentiment'] = test_df['polarity'].apply(lambda x: 0 if x == 0 else 1)
            
            # Basic preprocessing to match pipeline format
            test_df['text_processed'] = test_df['text'].str.lower().str.strip()
            test_df['text_length'] = test_df['text'].str.len()
            test_df['processed_length'] = test_df['text_processed'].str.len()
            test_df['token_count'] = test_df['text_processed'].str.split().str.len()
            
            logger.info(f"Loaded {len(test_df)} test samples")
            logger.info(f"Test set class distribution:\n{test_df['sentiment'].value_counts()}")
            
            return test_df
            
        except Exception as e:
            logger.error(f"Failed to load test data: {e}")
            raise
            
    def prepare_test_features(self, test_df: pd.DataFrame, tokenizer: Tokenizer, 
                            max_length: int = 100, numeric_features_dim: int = 0) -> Tuple:
        """
        Prepare test data features matching training format.
        
        Args:
            test_df: Test DataFrame
            tokenizer: Trained tokenizer from model
            max_length: Maximum sequence length
            numeric_features_dim: Number of numeric features
            
        Returns:
            Tuple of prepared test features
        """
        logger.info("Preparing test features...")
        
        # Use processed text
        text_column = 'text_processed' if 'text_processed' in test_df.columns else 'text'
        texts = test_df[text_column].fillna('').astype(str)
        
        # Convert to sequences using the trained tokenizer
        sequences = tokenizer.texts_to_sequences(texts)
        X_text = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
        
        # Prepare labels
        y_test = test_df['sentiment'].values.astype(np.float32)
        
        # Prepare numeric features if model uses them
        X_numeric = None
        if numeric_features_dim > 0:
            # Create dummy numeric features for test data
            # In production, you'd run the full feature engineering pipeline
            numeric_features = []
            
            # Basic features we can calculate
            if 'text_length' in test_df.columns:
                numeric_features.append(test_df['text_length'].fillna(0))
            if 'token_count' in test_df.columns:
                numeric_features.append(test_df['token_count'].fillna(0))
                
            # Add dummy values for missing features
            while len(numeric_features) < numeric_features_dim:
                numeric_features.append(np.zeros(len(test_df)))
                
            X_numeric = np.column_stack(numeric_features[:numeric_features_dim]).astype(np.float32)
            logger.info(f"Created {X_numeric.shape[1]} numeric features for test data")
            
        return X_text, X_numeric, y_test
        
    def evaluate_model(self, model_path: str, test_data: Tuple, model_name: str) -> Dict:
        """
        Evaluate a single model on test data with TPU optimization.
        
        Args:
            model_path: Path to saved model
            test_data: Prepared test data tuple
            model_name: Name of the model
            
        Returns:
            Dict: Evaluation metrics
        """
        logger.info(f"Evaluating {model_name} on test data...")
        
        try:
            # Load model with strategy scope
            with self.env_manager.strategy.scope():
                model = tf.keras.models.load_model(model_path)
                
            # Prepare test inputs
            if len(test_data) == 3:  # Has numeric features
                X_text, X_numeric, y_test = test_data
                test_inputs = [X_text, X_numeric]
            else:
                X_text, y_test = test_data
                test_inputs = X_text
                
            # Create optimized dataset for TPU
            if self.env_manager.device_type == 'TPU':
                test_dataset = self._create_tpu_dataset(test_inputs, y_test)
                
                # Evaluate with TPU strategy
                with self.env_manager.strategy.scope():
                    eval_results = model.evaluate(
                        test_dataset,
                        verbose=1,
                        return_dict=True
                    )
                    
                    # Get predictions for detailed metrics
                    predictions = model.predict(test_dataset, verbose=1)
                    
            else:
                # Regular evaluation for GPU/CPU
                eval_results = model.evaluate(
                    test_inputs, y_test,
                    batch_size=self.test_batch_size,
                    verbose=1,
                    return_dict=True
                )
                
                predictions = model.predict(
                    test_inputs,
                    batch_size=self.test_batch_size,
                    verbose=1
                )
                
            # Calculate additional metrics
            y_pred = (predictions > 0.5).astype(int).flatten()
            
            # Classification report
            from sklearn.metrics import classification_report, confusion_matrix
            class_report = classification_report(y_test, y_pred, output_dict=True)
            conf_matrix = confusion_matrix(y_test, y_pred)
            
            # Compile results
            results = {
                'model_name': model_name,
                'test_loss': eval_results.get('loss', None),
                'test_accuracy': eval_results.get('accuracy', None),
                'test_auc': eval_results.get('auc', None),
                'classification_report': class_report,
                'confusion_matrix': conf_matrix.tolist(),
                'test_samples': len(y_test),
                'batch_size_used': self.test_batch_size
            }
            
            # Log summary
            logger.info(f"{model_name} Test Results:")
            logger.info(f"  Loss: {results['test_loss']:.4f}")
            logger.info(f"  Accuracy: {results['test_accuracy']:.4f}")
            if results['test_auc']:
                logger.info(f"  AUC: {results['test_auc']:.4f}")
            logger.info(f"  Precision: {class_report['weighted avg']['precision']:.4f}")
            logger.info(f"  Recall: {class_report['weighted avg']['recall']:.4f}")
            logger.info(f"  F1-Score: {class_report['weighted avg']['f1-score']:.4f}")
            
            return results
            
        except Exception as e:
            logger.error(f"Failed to evaluate {model_name}: {e}")
            return {'model_name': model_name, 'error': str(e)}
            
    def _create_tpu_dataset(self, inputs, labels, shuffle: bool = False):
        """
        Create TPU-optimized dataset with maximum batch size.
        
        Args:
            inputs: Input data (text and/or numeric features)
            labels: Target labels
            shuffle: Whether to shuffle data
            
        Returns:
            tf.data.Dataset optimized for TPU
        """
        # Create dataset from tensor slices
        if isinstance(inputs, list):
            dataset = tf.data.Dataset.from_tensor_slices((tuple(inputs), labels))
        else:
            dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
            
        if shuffle:
            dataset = dataset.shuffle(buffer_size=len(labels))
            
        # Batch with TPU-optimized size and prefetch
        dataset = dataset.batch(self.test_batch_size, drop_remainder=False)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        
        # Cache in memory for TPU (we have 173GB RAM)
        dataset = dataset.cache()
        
        return dataset
        
    def visualize_results(self, results: List[Dict], save_path: str = '/content/'):
        """
        Create visualization of test results.
        
        Args:
            results: List of evaluation results
            save_path: Path to save visualizations
        """
        logger.info("Creating test results visualization...")
        
        # Create comparison plot
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # Extract metrics for plotting
        model_names = [r['model_name'] for r in results if 'error' not in r]
        accuracies = [r['test_accuracy'] for r in results if 'error' not in r]
        losses = [r['test_loss'] for r in results if 'error' not in r]
        f1_scores = [r['classification_report']['weighted avg']['f1-score'] 
                    for r in results if 'error' not in r]
        
        # Plot 1: Accuracy comparison
        ax = axes[0, 0]
        bars = ax.bar(model_names, accuracies, color='skyblue', edgecolor='navy', alpha=0.7)
        ax.set_ylabel('Accuracy', fontsize=12)
        ax.set_title('Model Accuracy on Test Set', fontsize=14, fontweight='bold')
        ax.set_ylim(0, 1)
        # Add value labels
        for bar, acc in zip(bars, accuracies):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                   f'{acc:.3f}', ha='center', va='bottom', fontsize=10)
        
        # Plot 2: Loss comparison
        ax = axes[0, 1]
        bars = ax.bar(model_names, losses, color='lightcoral', edgecolor='darkred', alpha=0.7)
        ax.set_ylabel('Loss', fontsize=12)
        ax.set_title('Model Loss on Test Set', fontsize=14, fontweight='bold')
        # Add value labels
        for bar, loss in zip(bars, losses):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                   f'{loss:.3f}', ha='center', va='bottom', fontsize=10)
        
        # Plot 3: F1-Score comparison
        ax = axes[1, 0]
        bars = ax.bar(model_names, f1_scores, color='lightgreen', edgecolor='darkgreen', alpha=0.7)
        ax.set_ylabel('F1-Score', fontsize=12)
        ax.set_title('Model F1-Score on Test Set', fontsize=14, fontweight='bold')
        ax.set_ylim(0, 1)
        # Add value labels
        for bar, f1 in zip(bars, f1_scores):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                   f'{f1:.3f}', ha='center', va='bottom', fontsize=10)
        
        # Plot 4: Confusion Matrix for best model
        ax = axes[1, 1]
        best_model = max(results, key=lambda x: x.get('test_accuracy', 0) if 'error' not in x else 0)
        if 'confusion_matrix' in best_model:
            conf_matrix = np.array(best_model['confusion_matrix'])
            im = ax.imshow(conf_matrix, cmap='Blues', aspect='auto')
            
            # Add colorbar
            plt.colorbar(im, ax=ax)
            
            # Add labels
            ax.set_xticks([0, 1])
            ax.set_yticks([0, 1])
            ax.set_xticklabels(['Negative', 'Positive'])
            ax.set_yticklabels(['Negative', 'Positive'])
            ax.set_xlabel('Predicted', fontsize=12)
            ax.set_ylabel('Actual', fontsize=12)
            ax.set_title(f'Confusion Matrix - {best_model["model_name"]}', fontsize=14, fontweight='bold')
            
            # Add text annotations
            for i in range(2):
                for j in range(2):
                    ax.text(j, i, str(conf_matrix[i, j]), 
                           ha='center', va='center', color='black', fontsize=14)
        
        plt.tight_layout()
        plt.savefig(f'{save_path}test_results_visualization.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        logger.info(f"Visualization saved to {save_path}test_results_visualization.png")

class GPUEnvironmentManager:
    """Manages GPU/TPU environment setup and optimization for Google Colab"""

    def __init__(self):
        self.device_type = self._detect_accelerator()
        self.strategy = self._setup_distribution_strategy()
        self.mixed_precision_enabled = False
        # self._configure_mixed_precision()
        self._log_environment_info()

        # if self.device_type == 'GPU':
            # self._setup_mixed_precision()

    def _detect_accelerator(self) -> str:
        """Detect available accelerator (GPU/TPU/CPU)"""
        try:
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            logger.info("TPU detected and initialized")
            return 'TPU'
        except (ValueError, RuntimeError):
            pass

        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            logger.info(f"GPU detected: {len(gpus)} device(s)")
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            return 'GPU'

        logger.warning("No accelerator detected, using CPU")
        return 'CPU'

    def _setup_distribution_strategy(self):
        """Setup distribution strategy based on hardware"""
        if self.device_type == 'TPU':
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
            strategy = tf.distribute.TPUStrategy(tpu)
            logger.info(f"Using TPU strategy with {strategy.num_replicas_in_sync} replicas")
        elif self.device_type == 'GPU':
            strategy = tf.distribute.MirroredStrategy()
            logger.info(f"Using MirroredStrategy with {strategy.num_replicas_in_sync} replicas")
        else:
            strategy = tf.distribute.get_strategy()
            logger.info("Using default strategy (CPU)")

        return strategy

    def _setup_mixed_precision(self):
        """Setup mixed precision training for faster GPU training"""
        try:
            policy = tf.keras.mixed_precision.Policy('mixed_float16')
            tf.keras.mixed_precision.set_global_policy(policy)
            self.mixed_precision_enabled = True
            logger.info("Mixed precision training enabled (float16)")
        except Exception as e:
            logger.warning(f"Could not enable mixed precision: {e}")

    def _log_environment_info(self):
        """Log environment information"""
        logger.info("=== GPU/TPU Environment Information ===")
        logger.info(f"TensorFlow version: {tf.__version__}")
        logger.info(f"Detected accelerator: {self.device_type}")

        if self.device_type == 'GPU':
            gpus = tf.config.list_physical_devices('GPU')
            for i, gpu in enumerate(gpus):
                logger.info(f"GPU {i}: {gpu}")

        logger.info("=" * 50)

    def get_optimal_batch_size(self, base_batch_size: int = 16384) -> int:
    """
    Calculate optimal batch size based on hardware - 
    optimized for v6e1 TPU
    """
    
    if self.device_type == 'TPU':
        # v6e1 TPU with 173GB RAM can handle massive batches
        # Use power of 2 for optimal TPU performance
        return 65536  # 64K for training
    elif self.device_type == 'GPU':
        return base_batch_size * max(1, self.strategy.num_replicas_in_sync)
    else:
        return max(16, base_batch_size // 2)


class PipelineOptimizedModel:
    """
    Base class for pipeline-compatible GPU/TPU optimized deep learning models
    Works with features from MICAP preprocessing pipeline
    """

    def __init__(self, env_manager: GPUEnvironmentManager,
                 max_words: int = 10000, max_length: int = 100):
        self.env_manager = env_manager
        self.max_words = max_words
        self.max_length = max_length
        self.model = None
        self.tokenizer = None
        self.history = None
        self.training_time = 0
        self.numeric_features_dim = 0

    def prepare_data(self, texts: np.ndarray, labels: np.ndarray,
                    numeric_features: Optional[np.ndarray] = None,
                    validation_split: float = 0.2) -> Tuple:
        """
        Prepare text and numeric features for training.

        Args:
            texts: Array of text data
            labels: Array of labels
            numeric_features: Optional array of numeric features from pipeline
            validation_split: Validation split ratio

        Returns:
            Tuple of prepared datasets
        """
        logger.info("Preparing data for pipeline-compatible training...")

        # Initialize tokenizer
        self.tokenizer = Tokenizer(
            num_words=self.max_words,
            oov_token='<OOV>',
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
        )

        # Fit tokenizer
        self.tokenizer.fit_on_texts(texts)

        # Convert to sequences
        sequences = self.tokenizer.texts_to_sequences(texts)
        X_text = pad_sequences(sequences, maxlen=self.max_length,
                              padding='post', truncating='post')

        # Prepare numeric features if available
        X_numeric = None
        if numeric_features is not None:
            X_numeric = numeric_features.astype(np.float32)
            self.numeric_features_dim = X_numeric.shape[1]
            logger.info(f"Using {self.numeric_features_dim} numeric features from pipeline")

        y = labels.astype(np.float32)

        # Split data
        if X_numeric is not None:
            X_text_train, X_text_val, X_num_train, X_num_val, y_train, y_val = train_test_split(
                X_text, X_numeric, y, test_size=validation_split,
                random_state=42, stratify=y
            )

            return (X_text_train, X_num_train, y_train), (X_text_val, X_num_val, y_val)
        else:
            X_text_train, X_text_val, y_train, y_val = train_test_split(
                X_text, y, test_size=validation_split,
                random_state=42, stratify=y
            )

            return (X_text_train, y_train), (X_text_val, y_val)

    def _get_optimizer(self, learning_rate: float = 0.001):
        """Get optimized optimizer"""
        if self.env_manager.device_type == 'TPU':
            optimizer = optimizers.Adam(learning_rate=learning_rate * 2)
        else:
            optimizer = optimizers.Adam(learning_rate=learning_rate)

        if self.env_manager.mixed_precision_enabled:
            optimizer = LossScaleOptimizer(optimizer)

        return optimizer

    def _get_callbacks(self, model_name: str, patience: int = 5):
        """Get training callbacks"""
        return [
            callbacks.EarlyStopping(
                monitor='val_loss', patience=patience,
                restore_best_weights=True, verbose=1
            ),
            callbacks.ReduceLROnPlateau(
                monitor='val_loss', factor=0.5,
                patience=max(2, patience // 2), min_lr=1e-7, verbose=1
            ),
            callbacks.ModelCheckpoint(
                filepath=f'/content/best_{model_name}_pipeline_model.h5',
                monitor='val_loss', save_best_only=True, verbose=1
            )
        ]


class PipelineLSTMModel(PipelineOptimizedModel):
    """LSTM model optimized for pipeline features"""

    def build_model(self, embedding_dim: int = 128, lstm_units: int = 64,
                   dropout_rate: float = 0.3):
        """Build LSTM model with optional numeric features integration"""
        logger.info("Building pipeline-compatible LSTM model...")

        with self.env_manager.strategy.scope():
            # Text input branch
            text_input = layers.Input(shape=(self.max_length,), name='text_input')

            # Embedding layer
            embedding = layers.Embedding(
                input_dim=self.max_words,
                output_dim=embedding_dim,
                input_length=self.max_length,
                mask_zero=True,
                name='embedding'
            )(text_input)

            embedding = layers.SpatialDropout1D(dropout_rate * 0.5)(embedding)

            # Bidirectional LSTM layers
            lstm1 = layers.Bidirectional(
                layers.LSTM(lstm_units, dropout=dropout_rate,
                            return_sequences=True),
                name='bi_lstm_1'
            )(embedding)

            lstm2 = layers.Bidirectional(
                layers.LSTM(lstm_units // 2, dropout=dropout_rate,
                            return_sequences=False),
                name='bi_lstm_2'
            )(lstm1)

            # Text features
            text_features = layers.Dense(64, activation='relu', name='text_dense')(lstm2)
            text_features = layers.BatchNormalization()(text_features)
            text_features = layers.Dropout(dropout_rate)(text_features)

            # Combine with numeric features if available
            if self.numeric_features_dim > 0:
                # Numeric input branch
                numeric_input = layers.Input(shape=(self.numeric_features_dim,), name='numeric_input')
                numeric_features = layers.Dense(32, activation='relu', name='numeric_dense')(numeric_input)
                numeric_features = layers.BatchNormalization()(numeric_features)
                numeric_features = layers.Dropout(dropout_rate * 0.5)(numeric_features)

                # Combine text and numeric features
                combined = layers.Concatenate(name='combine_features')([text_features, numeric_features])
                inputs = [text_input, numeric_input]
            else:
                combined = text_features
                inputs = text_input

            # Final classification layers
            dense = layers.Dense(32, activation='relu', name='final_dense')(combined)
            dense = layers.Dropout(dropout_rate * 0.5)(dense)

            output = layers.Dense(1, activation='sigmoid', name='output')(dense)

            model = models.Model(inputs=inputs, outputs=output, name='PipelineLSTM')

        self.model = model

        # Compile model
        optimizer = self._get_optimizer()
        self.model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
        )

        logger.info("LSTM Model Architecture:")
        self.model.summary(print_fn=logger.info)

        return model

    def train(self, train_data: Tuple, val_data: Tuple, epochs: int = 10):
        """Train the LSTM model"""
        logger.info("Training pipeline LSTM model...")

        callbacks_list = self._get_callbacks('LSTM')

        start_time = time.time()

        # Prepare training data
        if self.numeric_features_dim > 0:
            X_text_train, X_num_train, y_train = train_data
            X_text_val, X_num_val, y_val = val_data

            train_inputs = [X_text_train, X_num_train]
            val_inputs = [X_text_val, X_num_val]
        else:
            X_text_train, y_train = train_data
            X_text_val, y_val = val_data

            train_inputs = X_text_train
            val_inputs = X_text_val

        batch_size = self.env_manager.get_optimal_batch_size()

        # with self.env_manager.strategy.scope():
        #     self.history = self.model.fit(
        #         train_inputs, y_train,
        #         batch_size=batch_size,
        #         epochs=epochs,
        #         validation_data=(val_inputs, y_val),
        #         callbacks=callbacks_list,
        #         verbose=1
        #     )
        def make_ds(x_text, x_num, y, shuffle=True):
            inputs = (x_text, x_num) if x_num is not None else x_text
            ds = tf.data.Dataset.from_tensor_slices((inputs, y))
            if shuffle:
                ds = ds.shuffle(100_000)
            return ds.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

        train_ds = make_ds(*train_data, shuffle=True)
        val_ds   = make_ds(*val_data,   shuffle=False)

        with self.env_manager.strategy.scope():
            self.history = self.model.fit(
                train_ds,
                epochs=epochs,
                validation_data=val_ds,
                callbacks=callbacks_list,
                verbose=1
            )

        self.training_time = time.time() - start_time
        logger.info(f"Training completed in {self.training_time:.2f} seconds")

        return self.history.history


def evaluate_pipeline_models_gpu(data_path: str,
                                sample_fraction: float = 1.0,
                                epochs: int = 10) -> Dict:
    """
    Evaluate models using MICAP pipeline data with GPU/TPU optimization.

    Args:
        data_path: Path to processed pipeline data (parquet)
        sample_fraction: Fraction of data to use
        epochs: Number of training epochs

    Returns:
        Dict: Evaluation results
    """
    logger.info("Starting pipeline-compatible GPU evaluation...")

    # Initialize components
    env_manager = GPUEnvironmentManager()
    data_loader = PipelineDataLoader()

    # Load pipeline data
    df = data_loader.load_pipeline_data(data_path, sample_fraction)

    # Prepare features
    texts, labels, numeric_features, feature_info = data_loader.prepare_features_for_training(df)

    logger.info(f"Loaded pipeline data: {feature_info}")

    # Initialize models
    models_to_evaluate = [
        ('Pipeline_LSTM', PipelineLSTMModel)
    ]

    results = {}

    for model_name, ModelClass in models_to_evaluate:
        logger.info(f"\n{'='*60}")
        logger.info(f"Training {model_name}")
        logger.info(f"{'='*60}")

        try:
            # Initialize model
            model = ModelClass(env_manager)

            # Prepare data
            train_data, val_data = model.prepare_data(
                texts, labels, numeric_features
            )

            # Build model
            model.build_model()

            # Train model
            history = model.train(train_data, val_data, epochs=epochs)

            # Evaluate
            if model.numeric_features_dim > 0:
                val_inputs = [val_data[0], val_data[1]]
                val_labels = val_data[2]
            else:
                val_inputs = val_data[0]
                val_labels = val_data[1]

            eval_results = model.model.evaluate(val_inputs, val_labels, verbose=0)

            metrics = {}
            for i, metric_name in enumerate(model.model.metrics_names):
                metrics[metric_name] = eval_results[i]
            metrics['training_time'] = model.training_time

            results[model_name] = {
                'metrics': metrics,
                'history': history,
                'feature_info': feature_info
            }

            # Save model
            model.model.save(f'/content/{model_name.lower()}_model.h5')
            logger.info(f"Model saved as {model_name.lower()}_model.h5")

        except Exception as e:
            logger.error(f"Failed to train {model_name}: {e}")
            results[model_name] = {'error': str(e)}

    # Save results
    with open('pipeline_model_results.json', 'w') as f:
        json.dump(results, f, indent=2, default=str)

    logger.info("Pipeline evaluation completed!")
    return results


# Usage examples for Google Colab
# if __name__ == "__main__":
#     """
#     Example usage in Google Colab:

#     # Option 1: Use your existing pipeline data (RECOMMENDED)
#     results = evaluate_pipeline_models_gpu(
#         data_path='/content/pipeline_features.parquet',  # Upload your processed parquet file
#         sample_fraction=1.0,
#         epochs=10
#     )

#     # Option 2: If you have PySpark setup in Colab (advanced)
#     # First run your preprocessing pipeline to create the parquet file
#     # Then use the above approach
#     """

#     results = evaluate_pipeline_models_gpu(
#         'part-00000.snappy.parquet',
#         sample_fraction=1.0,
#         epochs=5
#     )
#     results = evaluate_pipeline_models_gpu(
#         'part-00001.snappy.parquet',
#         sample_fraction=1.0,
#         epochs=5
#     )
#     results = evaluate_pipeline_models_gpu(
#         'part-00002.snappy.parquet',
#         sample_fraction=1.0,
#         epochs=5
#     )
#     results = evaluate_pipeline_models_gpu(
#         'part-00003.snappy.parquet',
#         sample_fraction=1.0,
#         epochs=5
#     )

#     logger.info("Pipeline-compatible GPU deep learning models ready!")
#     logger.info("Upload your 'pipeline_features.parquet' file to /content/ and run evaluation.")

def test_pipeline_models_on_dataset(
    model_files: List[str],
    test_csv_path: str = '/content/testdata.manual.2009.06.14.csv',
    save_results: bool = True
) -> Dict:
    """
    Test trained models on the Sentiment140 test dataset with TPU optimization.
    
    Args:
        model_files: List of model file paths to test
        test_csv_path: Path to test CSV file
        save_results: Whether to save results to file
        
    Returns:
        Dict: Comprehensive test results
    """
    logger.info("="*60)
    logger.info("STARTING MODEL EVALUATION ON TEST DATASET")
    logger.info(f"TPU-Optimized for v6e1 with 173GB RAM")
    logger.info("="*60)
    
    # Initialize environment
    env_manager = GPUEnvironmentManager()
    data_loader = PipelineDataLoader()
    evaluator = TestDataEvaluator(env_manager, data_loader)
    
    # Load test data
    try:
        test_df = evaluator.load_test_data(test_csv_path)
    except:
        # Try alternative path if default fails
        logger.warning("Default test path failed, trying alternative...")
        test_df = evaluator.load_test_data('/content/drive/MyDrive/Uni/BD/testdata.manual.2009.06.14.csv')
    
    # Store all results
    all_results = []
    
    # Evaluate each model
    for model_file in model_files:
        if not os.path.exists(model_file):
            logger.warning(f"Model file not found: {model_file}")
            continue
            
        try:
            # Extract model name from filename
            model_name = os.path.basename(model_file).replace('.h5', '').replace('best_', '').upper()
            
            logger.info(f"\n{'='*50}")
            logger.info(f"Evaluating: {model_name}")
            logger.info(f"{'='*50}")
            
            # Load model to get configuration
            with env_manager.strategy.scope():
                model = tf.keras.models.load_model(model_file)
                
                # Get tokenizer configuration (you might need to save/load this separately)
                # For now, we'll recreate it from the model's embedding layer
                embedding_layer = model.get_layer('embedding')
                max_words = embedding_layer.input_dim
                max_length = embedding_layer.input_length or 100
                
                # Check if model uses numeric features
                has_numeric = any('numeric_input' in layer.name for layer in model.layers)
                numeric_dim = 0
                if has_numeric:
                    numeric_input_layer = model.get_layer('numeric_input')
                    numeric_dim = numeric_input_layer.input_shape[-1]
                
            # Create tokenizer (in production, you'd save/load this with the model)
            tokenizer = Tokenizer(
                num_words=max_words,
                oov_token='<OOV>',
                filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
            )
            
            # Fit tokenizer on test data (in production, use saved tokenizer)
            tokenizer.fit_on_texts(test_df['text_processed'].fillna('').astype(str))
            
            # Prepare test features
            X_text, X_numeric, y_test = evaluator.prepare_test_features(
                test_df, tokenizer, max_length, numeric_dim
            )
            
            # Create test data tuple
            if X_numeric is not None:
                test_data = (X_text, X_numeric, y_test)
            else:
                test_data = (X_text, y_test)
            
            # Evaluate model
            results = evaluator.evaluate_model(model_file, test_data, model_name)
            all_results.append(results)
            
        except Exception as e:
            logger.error(f"Failed to evaluate {model_file}: {e}")
            all_results.append({
                'model_name': model_name,
                'error': str(e)
            })
    
    # Create visualizations
    if all_results:
        evaluator.visualize_results(all_results)
    
    # Save results
    if save_results:
        results_file = '/content/test_results_comprehensive.json'
        with open(results_file, 'w') as f:
            json.dump(all_results, f, indent=2, default=str)
        logger.info(f"Results saved to {results_file}")
    
    # Print summary
    logger.info("\n" + "="*60)
    logger.info("TEST EVALUATION SUMMARY")
    logger.info("="*60)
    
    for result in all_results:
        if 'error' not in result:
            logger.info(f"\n{result['model_name']}:")
            logger.info(f"  Test Accuracy: {result['test_accuracy']:.4f}")
            logger.info(f"  Test Loss: {result['test_loss']:.4f}")
            if result.get('test_auc'):
                logger.info(f"  Test AUC: {result['test_auc']:.4f}")
            logger.info(f"  F1-Score: {result['classification_report']['weighted avg']['f1-score']:.4f}")
    
    return all_results

# Usage example - add this at the very end of your notebook
if __name__ == "__main__":
    # After training your models, test them
    trained_models = [
        '/content/best_LSTM_pipeline_model.h5',
        # Add other model paths here as you implement CNN and Transformer
        # '/content/best_CNN_pipeline_model.h5',
        # '/content/best_Transformer_pipeline_model.h5'
    ]
    
    # Run comprehensive testing
    test_results = test_pipeline_models_on_dataset(
        model_files=trained_models,
        test_csv_path='/content/testdata.manual.2009.06.14.csv'
    )
    
    logger.info("\n✅ Testing completed! Check visualization and results file.")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7363 - auc: 0.0000e+00 - loss: 0.5106
Epoch 1: val_loss improved from inf to 0.05236, saving model to /content/best_LSTM_pipeline_model.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.7414 - auc: 0.0000e+00 - loss: 0.5045 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - val_loss: 0.0524 - learning_rate: 0.0010
Epoch 2/5
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9992 - auc: 0.0000e+00 - loss: 0.0520
Epoch 2: val_loss improved from 0.05236 to 0.04999, saving model to /content/best_LSTM_pipeline_model.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.9992 - auc: 0.0000e+00 - loss: 0.0515 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - val_loss: 0.0500 - learning_rate: 0.0010
Epoch 3/5
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9999 - auc: 0.0000e+00 - loss: 0.0138
Epoch 3: val_loss improved from 0.04999 to 0.03539, saving model to /content/best_LSTM_pipeline_model.h5


29/29 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.9999 - auc: 0.0000e+00 - loss: 0.0137 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - val_loss: 0.0354 - learning_rate: 0.0010
Epoch 4/5
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9999 - auc: 0.0000e+00 - loss: 0.0071
Epoch 4: val_loss improved from 0.03539 to 0.01963, saving model to /content/best_LSTM_pipeline_model.h5


...

25/25 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.6851 - auc: 0.0000e+00 - loss: 0.5551 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - val_loss: 0.1732 - learning_rate: 0.0010
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9966 - auc: 0.0000e+00 - loss: 0.0992
Epoch 2: val_loss improved from 0.17322 to 0.09303, saving model to /content/best_LSTM_pipeline_model.h5


25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.9967 - auc: 0.0000e+00 - loss: 0.0981 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - val_loss: 0.0930 - learning_rate: 0.0010
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9995 - auc: 0.0000e+00 - loss: 0.0233
Epoch 3: val_loss improved from 0.09303 to 0.07109, saving model to /content/best_LSTM_pipeline_model.h5


25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.9995 - auc: 0.0000e+00 - loss: 0.0232 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - val_loss: 0.0711 - learning_rate: 0.0010
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9999 - auc: 0.0000e+00 - loss: 0.0099
Epoch 4: val_loss improved from 0.07109 to 0.04289, saving model to /content/best_LSTM_pipeline_model.h5


25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.9999 - auc: 0.0000e+00 - loss: 0.0099 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - val_loss: 0.0429 - learning_rate: 0.0010
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 1.0000 - auc: 0.0000e+00 - loss: 0.0057
Epoch 5: val_loss improved from 0.04289 to 0.02277, saving model to /content/best_LSTM_pipeline_model.h5


25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 1.0000 - auc: 0.0000e+00 - loss: 0.0057 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - val_loss: 0.0228 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 5.
